In [ ]:
from util.models import RecommendResult, Dataset
from base_recommender import BaseRecommender
from collections import defaultdict
import numpy as np
import implicit
from scipy.sparse import lil_matrix
np.random.seed(0)

class IMFRecommender(BaseRecommender):
    """
    IMF (Implicit Matrix Factorization) 추천기
    - 명시적 평점(1~5) 대신 암묵적 피드백(봤다/안봤다, 0/1)을 사용
    - 평점 예측은 안 하고, 좋아할 아이템 추천만 함
    - ALS(교대최소제곱법)로 유저벡터 × 아이템벡터 학습
    """
    def recommend(self, dataset, **kwargs):
        factors = kwargs.get("factors",10)          # 잠재 요인 수 (유저/아이템 벡터 차원)
        minimum_num_rating = kwargs.get("minimum_num_rating",0)  # 최소 평가 수 필터
        n_epochs = kwargs.get("n_epochs",50)        # ALS 반복 횟수
        alpha = kwargs.get("alpha",1.0)             # 신뢰도 가중치 (c = 1 + alpha * r)

        # 1단계: 데이터 전처리 — 평점 4 이상만 추출 (좋아함 = 1)
        filtered_movielens_train = dataset.train.groupby("movie_id").filter(
            lambda x:len(x) >=minimum_num_rating    # 평가 수가 너무 적은 영화 제거
        )
        movielens_train_high_rating = filtered_movielens_train[filtered_movielens_train.rating>=4]

        # 2단계: ID → 행렬 인덱스 매핑 (유저/영화 ID는 연속이 아니라서 필요)
        unique_user_ids = sorted(movielens_train_high_rating.user_id.unique())
        unique_movie_ids = sorted(movielens_train_high_rating.movie_id.unique())
        user_id2index = dict(zip(unique_user_ids,range(len(unique_user_ids)) ))
        movie_id2index = dict(zip(unique_movie_ids,range(len(unique_movie_ids)) ))

        # 3단계: 희소 행렬 생성 (유저 × 영화)
        # 평점 4 이상인 칸만 1*alpha, 나머지는 0
        movielens_matrix = lil_matrix((len(unique_user_ids),len(unique_movie_ids)))
        for i, row in movielens_train_high_rating.iterrows():
            user_index = user_id2index[row["user_id"]]
            movie_index = movie_id2index[row["movie_id"]]
            movielens_matrix[user_index, movie_index] = 1.0 * alpha
        movielens_matrix = movielens_matrix.tocsr()  # lil → csr 변환 (implicit 라이브러리가 요구)

        # 4단계: ALS 모델 학습
        # 유저벡터(Xᵤ)와 아이템벡터(Yᵢ)를 번갈아 최적화하며 학습
        model = implicit.als.AlternatingLeastSquares(
            factors = factors,
            iterations=n_epochs, calculate_training_loss=True, random_state=1
        )
        model.fit(movielens_matrix)

        # 5단계: 전체 유저에 대해 top-10 아이템 추천
        # recommend_all은 각 유저별로 점수 높은 아이템 인덱스를 반환
        recommendations = model.recommend_all(movielens_matrix)
        pred_love_items = defaultdict(list)
        for user_id, user_index in user_id2index.items():
            movie_indexes = recommendations[user_index,:]   # 해당 유저의 추천 아이템 인덱스들
            for movie_index in movie_indexes:
                movie_id = unique_movie_ids[movie_index]     # 인덱스 → 실제 영화 ID로 변환
                pred_love_items[user_id].append(movie_id)

        # IMF는 평점 예측을 안 하므로 실제 평점을 그대로 넘김 (RMSE는 의미 없음)
        return RecommendResult(dataset.test.rating, pred_love_items)

if __name__ == "__main__":
    IMFRecommender().eval()